In [1]:
#Ensure you update this
match_link="https://www.espncricinfo.com/series/8048/scorecard/1237181/delhi-capitals-vs-mumbai-indians-final-indian-premier-league-2020-21"
match_no=60
innings=['DC','MI']
man_of_the_match="%Rohit%Sharma%"
winning_team="MI"


In [2]:
teams="60	MI vs DC	Govardhan	Surya Kumar Yadav	Ishan Kishan	Shikhar Dhawan	Shreyas Iyer	Kieron Pollard✈	Marcus Stoinis✈	Quinton De Kock✈	Jasprit Bumrah	Kagiso Rabada✈	Axar Patel	Krunal Pandya	Surya Kumar Yadav	Shikhar Dhawan	MI	\
60	MI vs DC	Sowmya	Rohit Sharma	Shikhar Dhawan	Surya Kumar Yadav	Ishan Kishan	Kieron Pollard✈	Marcus Stoinis✈	Quinton De Kock✈	Kagiso Rabada✈	Jasprit Bumrah	Axar Patel	Rahul Chahar	Ishan Kishan	Jasprit Bumrah	MI	\
60	MI vs DC	Avani	Shikhar Dhawan	Surya Kumar Yadav	Rohit Sharma	Shreyas Iyer	Marcus Stoinis✈	Axar Patel	Quinton De Kock✈	Kagiso Rabada✈	Jasprit Bumrah	Rahul Chahar	Anrich Nortje✈	Shikhar Dhawan	Jasprit Bumrah	MI	\
60	MI vs DC	Deepak	Ishan Kishan	Shikhar Dhawan	Surya Kumar Yadav	Shimron Hetmyer✈	Rohit Sharma	Marcus Stoinis✈	Quinton De Kock✈	Hardik Pandya	Trent Boult✈	Jasprit Bumrah	Ravichandran Ashwin	Rohit Sharma	Rohit Sharma	MI	\
60	MI vs DC	Aditya	Ishan Kishan	Shikhar Dhawan	Surya Kumar Yadav	Shreyas Iyer	Ravichandran Ashwin	Rohit Sharma	Quinton De Kock✈	Jasprit Bumrah	Marcus Stoinis✈	Kagiso Rabada✈	Trent Boult✈	Surya Kumar Yadav	Jasprit Bumrah	MI	\
60	MI vs DC	Mitesh	Shikhar Dhawan	Ishan Kishan	Surya Kumar Yadav	Shimron Hetmyer✈	Rohit Sharma	Marcus Stoinis✈	Quinton De Kock✈	Jasprit Bumrah	Kagiso Rabada✈	Hardik Pandya	Ravichandran Ashwin	Surya Kumar Yadav	Ravichandran Ashwin	DC	\
60	MI vs DC	Shriyansh	Quinton De Kock✈	Shikhar Dhawan	Shreyas Iyer	Surya Kumar Yadav	Kieron Pollard✈	Rohit Sharma	Ishan Kishan	Kagiso Rabada✈	Jasprit Bumrah	Hardik Pandya	Trent Boult✈	Quinton De Kock✈	Jasprit Bumrah	MI	"
teams=teams.replace("✈","")

In [3]:
#Connect to cricinfo website and get the response into soup using beautiful soup library
import pandas as pd
import html5lib

all_queries_list=[]


#	Player_Name	is_out	R	B	4s	6s	SR	Team	4s_points	6s_points	runs_points	runs_bonus_points	sr_points	total_batting_points

#extract 1st innings batting score
dfbat1 = pd.read_html(match_link,skiprows=0)[0]
dfbat1['Team']=innings[0]
dfbat1['Team_Field']=innings[1]

#extract 2nd innings batting score (logic to run when only one innings is done)
try:
    dfbat2 = pd.read_html(match_link,skiprows=0)[2]
    dfbat2['Team']=innings[1]
    dfbat2['Team_Field']=innings[0]
    #combine 2 innings batting score
    final_batting_scorecard=pd.concat([dfbat1, dfbat2],sort=True)
except:
    final_batting_scorecard=dfbat1

#final_batting_scorecard=pd.concat([dfbat1, dfbat2])

#Remove Unnecessary rows
final_batting_scorecard = final_batting_scorecard[final_batting_scorecard['BATSMEN'].notna()]
final_batting_scorecard=final_batting_scorecard[final_batting_scorecard.BATSMEN != 'TOTAL']
final_batting_scorecard=final_batting_scorecard[final_batting_scorecard.BATSMEN != 'Extras']
final_batting_scorecard=final_batting_scorecard.dropna(axis=1, how='all')

#Rename Columns
final_batting_scorecard=final_batting_scorecard.rename(columns={'BATSMEN': 'Player_Name'})
final_batting_scorecard=final_batting_scorecard.rename(columns={'Unnamed: 1': 'Dismissal_Detail'})

#Dervie Not out details
final_batting_scorecard.loc[final_batting_scorecard['Dismissal_Detail']=='not out', 'is_out'] ='N'
final_batting_scorecard.loc[final_batting_scorecard['Dismissal_Detail']=='retired hurt', 'is_out'] ='N'
final_batting_scorecard.loc[final_batting_scorecard['Dismissal_Detail']!='not out', 'is_out'] ='Y'


#Dervie batting pos
final_batting_scorecard=final_batting_scorecard.reset_index()
final_batting_scorecard['batting_pos'] = final_batting_scorecard.index

#Clean Player Name
final_batting_scorecard['Player_Name'] = final_batting_scorecard['Player_Name'].str.replace('\(c\)','')
final_batting_scorecard['Player_Name'] = final_batting_scorecard['Player_Name'].str.strip()
final_batting_scorecard['Player_Name'] = final_batting_scorecard['Player_Name'].str.replace('†','')
final_batting_scorecard['Player_Name'] = final_batting_scorecard['Player_Name'].str.strip()

#Fix scenario where batsmen didn't score any run
try:
    final_batting_scorecard['SR'] = final_batting_scorecard['SR'].str.replace('-','0')
except:
    pass


In [4]:
#Update data types
final_batting_scorecard['R'] = final_batting_scorecard['R'].astype(int)
final_batting_scorecard['B'] = final_batting_scorecard['B'].astype(int)
final_batting_scorecard['4s'] = final_batting_scorecard['4s'].astype(int)
final_batting_scorecard['6s'] = final_batting_scorecard['6s'].astype(int)
final_batting_scorecard['SR'] = final_batting_scorecard['SR'].astype(float)
final_batting_scorecard

,index,4s,6s,B,Player_Name,R,SR,Team,Team_Field,Dismissal_Detail,is_out,batting_pos
0,0,0,0,1,MP Stoinis,0,0.00,DC,MI,c †de Kock b Boult,Y,0
1,2,3,0,13,S Dhawan,15,115.38,DC,MI,b J Yadav,Y,1
2,4,0,0,4,AM Rahane,2,50.00,DC,MI,c †de Kock b Boult,Y,2
3,6,6,2,50,SS Iyer,65,130.00,DC,MI,not out,N,3
4,8,4,2,38,RR Pant,56,147.36,DC,MI,c HH Pandya b Coulter-Nile,Y,4
5,10,1,0,5,SO Hetmyer,5,100.00,DC,MI,c Coulter-Nile b Boult,Y,5
6,12,1,0,9,AR Patel,9,100.00,DC,MI,c sub (AS Roy) b Coulter-Nile,Y,6
7,14,0,0,0,K Rabada,0,0.00,DC,MI,run out (SA Yadav/Coulter-Nile),Y,7
8,0,5,4,51,RG Sharma,68,133.33,MI,DC,c sub (Lalit Yadav) b Nortje,Y,8
9,2,3,1,12,Q de Kock,20,166.66,MI,DC,c †Pant b Stoinis,Y,9


In [5]:
import operator

final_batting_scorecard['4s_points']=final_batting_scorecard['4s']*5
final_batting_scorecard['6s_points']=final_batting_scorecard['6s']*10
final_batting_scorecard['runs_points']=0
final_batting_scorecard.loc[final_batting_scorecard['R'] >=10, 'runs_points'] =final_batting_scorecard['R']*1

final_batting_scorecard['runs_bonus_points']=0
final_batting_scorecard.loc[final_batting_scorecard['R'] >=50, 'runs_bonus_points'] = 25
final_batting_scorecard.loc[final_batting_scorecard['R'] >=100, 'runs_bonus_points'] = 75
final_batting_scorecard.loc[operator.and_(final_batting_scorecard['R']==0, final_batting_scorecard['is_out']!='N'), 'runs_bonus_points'] =-20
final_batting_scorecard['sr_points']=0
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=0 ),final_batting_scorecard['SR']<50), 'sr_points'] =-30
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=50 ),final_batting_scorecard['SR']<75), 'sr_points'] =-20
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=75 ),final_batting_scorecard['SR']<100), 'sr_points'] =-10
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=100), final_batting_scorecard['SR']<125), 'sr_points'] =0
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=125), final_batting_scorecard['SR']<175), 'sr_points'] =10
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=175), final_batting_scorecard['SR']<250), 'sr_points'] =20
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=250), final_batting_scorecard['SR']<1000), 'sr_points'] =30
final_batting_scorecard['total_batting_points']=final_batting_scorecard['4s_points']+final_batting_scorecard['6s_points']+final_batting_scorecard['runs_points']+final_batting_scorecard['sr_points']+final_batting_scorecard['runs_bonus_points']
final_batting_scorecard=final_batting_scorecard.sort_values(by=['Player_Name'])
final_batting_scorecard=final_batting_scorecard.reset_index()
del final_batting_scorecard['index']
del final_batting_scorecard['level_0']


final_batting_scorecard

,4s,6s,B,Player_Name,R,SR,Team,Team_Field,Dismissal_Detail,is_out,batting_pos,4s_points,6s_points,runs_points,runs_bonus_points,sr_points,total_batting_points
0,0,0,4,AM Rahane,2,50.00,DC,MI,c †de Kock b Boult,Y,2,0,0,0,0,0,0
1,1,0,9,AR Patel,9,100.00,DC,MI,c sub (AS Roy) b Coulter-Nile,Y,6,5,0,0,0,0,5
2,0,0,5,HH Pandya,3,60.00,MI,DC,c Rahane b Nortje,Y,13,0,0,0,0,0,0
3,3,1,19,Ishan Kishan,33,173.68,MI,DC,not out,N,11,15,10,33,0,10,68
4,0,0,0,K Rabada,0,0.00,DC,MI,run out (SA Yadav/Coulter-Nile),Y,7,0,0,0,-20,0,-20
5,2,0,4,KA Pollard,9,225.00,MI,DC,b Rabada,Y,12,10,0,0,0,0,10
6,0,0,1,KH Pandya,1,100.00,MI,DC,not out,N,14,0,0,0,0,0,0
7,0,0,1,MP Stoinis,0,0.00,DC,MI,c †de Kock b Boult,Y,0,0,0,0,-20,0,-20
8,3,1,12,Q de Kock,20,166.66,MI,DC,c †Pant b Stoinis,Y,9,15,10,20,0,10,55
9,5,4,51,RG Sharma,68,133.33,MI,DC,c sub (Lalit Yadav) b Nortje,Y,8,25,40,68,25,10,168


In [6]:


for index, row in final_batting_scorecard.iterrows():
    first, *middle, last = str(row["Player_Name"]).split()
    player_name_short=first[0].lower()+"%"+last.lower()
    all_queries_list.append("Update ipl_match_stats_new set batting_pos="+str(row["batting_pos"])+
           " where lower(player_name) like '"+player_name_short+"%' and team_name='"+row["Team"]+
           "' and match_no="+str(match_no)+";")

for index, row in final_batting_scorecard.iterrows():
    all_queries_list.append("Update ipl_match_stats_new set runs_scored="+str(row["R"])+
                            ",balls_faced="+str(row["B"])+
                            ",fours_scored="+str(row["4s"])+
                            ",sixes_scored="+str(row["6s"])+
                            ",is_out='"+str(row["is_out"])+
                            "' where batting_pos="+str(row["batting_pos"])+" and match_no="+str(match_no)+";")
    


In [7]:
#Player_Name	Team_BOWL	O	M	R	W	Econ	0s	4s	6s	WD	NB	wicket_points	wicket_bonus_points	maiden_points	dot_ball_points	econ_points	total_bowling_points
#extract 1st innings bowling score
dfbowl1 = pd.read_html(match_link,skiprows=0)[1]
dfbowl1['Team_BOWL']=innings[1]

#extract 2nd innings bowling score (logic to run when only one innings is done)
try:
    dfbowl2 = pd.read_html(match_link,skiprows=0)[3]
    dfbowl2['Team_BOWL']=innings[0]
    #combine bowling scores
    final_bowling_scorecard=pd.concat([dfbowl1, dfbowl2],sort=True)
except:
    print("except")
    final_bowling_scorecard=dfbowl1

#final_bowling_scorecard=pd.concat([dfbowl1, dfbowl2])
    
final_bowling_scorecard=final_bowling_scorecard.dropna(axis=1, how='all')

#cleanup
try:
    for i in range(0, 10):
        del final_bowling_scorecard[ii]
except:
    pass

final_bowling_scorecard = final_bowling_scorecard[final_bowling_scorecard['BOWLING'].notna()]


#Remove unncessary rows
final_bowling_scorecard=final_bowling_scorecard[final_bowling_scorecard.BOWLING != 'TOTAL']

#Rename columns
final_bowling_scorecard=final_bowling_scorecard.rename(columns={'BOWLING': 'Player_Name'})
final_bowling_scorecard=final_bowling_scorecard.rename(columns={'ECON': 'Econ'})


#Update data types
final_bowling_scorecard['O'] = final_bowling_scorecard['O'].astype(float)
final_bowling_scorecard['M'] = final_bowling_scorecard['M'].astype(int)
final_bowling_scorecard['R'] = final_bowling_scorecard['R'].astype(int)
final_bowling_scorecard['W'] = final_bowling_scorecard['W'].astype(int)
final_bowling_scorecard['Econ'] = final_bowling_scorecard['Econ'].astype(float)
final_bowling_scorecard['0s'] = final_bowling_scorecard['0s'].astype(int)
final_bowling_scorecard['4s'] = final_bowling_scorecard['4s'].astype(int)
final_bowling_scorecard['6s'] = final_bowling_scorecard['6s'].astype(int)
final_bowling_scorecard['WD'] = final_bowling_scorecard['WD'].astype(int)
final_bowling_scorecard['NB'] = final_bowling_scorecard['NB'].astype(int)

#cleanup
for i in range(0, 10):
    try:
        del final_bowling_scorecard['Unnamed: 11']
    except:
        pass
    try:
        del final_bowling_scorecard[i]
    except:
        pass
    try:
        del final_bowling_scorecard[str(i)]
    except:
        pass

final_bowling_scorecard

,0s,4s,6s,Player_Name,Econ,M,NB,O,R,Team_BOWL,W,WD
0,12,6,0,TA Boult,7.50,0,0,4.0,30,MI,3,0
1,8,3,0,JJ Bumrah,7.00,0,0,4.0,28,MI,0,0
2,7,1,0,J Yadav,6.25,0,0,4.0,25,MI,1,0
3,10,3,1,NM Coulter-Nile,7.25,0,0,4.0,29,MI,2,1
4,3,1,2,KH Pandya,10.00,0,0,3.0,30,MI,0,1
5,2,1,1,KA Pollard,13.00,0,0,1.0,13,MI,0,1
0,8,2,1,R Ashwin,7.00,0,0,4.0,28,DC,0,0
1,6,5,1,K Rabada,10.66,0,0,3.0,32,DC,1,0
2,6,3,1,A Nortje,9.37,0,0,2.4,25,DC,2,0
3,3,1,2,MP Stoinis,11.50,0,0,2.0,23,DC,1,0


In [8]:
final_bowling_scorecard['wicket_points']=final_bowling_scorecard['W']*25
final_bowling_scorecard['wicket_bonus_points']=0
final_bowling_scorecard.loc[final_bowling_scorecard['W'] >=3, 'wicket_bonus_points'] = 25
final_bowling_scorecard.loc[final_bowling_scorecard['W'] >=4,'wicket_bonus_points'] = 50
final_bowling_scorecard.loc[final_bowling_scorecard['W'] >=5,'wicket_bonus_points'] = 75
final_bowling_scorecard.loc[final_bowling_scorecard['W'] >=5,'wicket_bonus_points'] = 75
final_bowling_scorecard['maiden_points']=final_bowling_scorecard['M']*25
final_bowling_scorecard['dot_ball_points']=0
final_bowling_scorecard['dot_ball_points']=4*(final_bowling_scorecard['0s']-final_bowling_scorecard['4s']-1.5*final_bowling_scorecard['6s'])

final_bowling_scorecard['econ_points']=0
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>0 ),final_bowling_scorecard['Econ']<=4), 'econ_points'] =25
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>4 ),final_bowling_scorecard['Econ']<=6), 'econ_points'] =15
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>6 ),final_bowling_scorecard['Econ']<=8), 'econ_points'] =10
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>8 ),final_bowling_scorecard['Econ']<=10), 'econ_points'] =0
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>=10 ),final_bowling_scorecard['Econ']<12), 'econ_points'] =-10
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>=12 ),final_bowling_scorecard['Econ']<14), 'econ_points'] =-15
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>=14 ),final_bowling_scorecard['Econ']<100), 'econ_points'] =-25
final_bowling_scorecard['total_bowling_points']=final_bowling_scorecard['maiden_points']+final_bowling_scorecard['wicket_points']+final_bowling_scorecard['wicket_bonus_points']+final_bowling_scorecard['dot_ball_points']+final_bowling_scorecard['econ_points']
final_bowling_scorecard=final_bowling_scorecard.sort_values(by=['Player_Name'])
final_bowling_scorecard=final_bowling_scorecard.reset_index()
final_bowling_scorecard['bowling_pos'] = final_bowling_scorecard.index
del final_bowling_scorecard['index']
final_bowling_scorecard

,0s,4s,6s,Player_Name,Econ,M,NB,O,R,Team_BOWL,W,WD,wicket_points,wicket_bonus_points,maiden_points,dot_ball_points,econ_points,total_bowling_points,bowling_pos
0,6,3,1,A Nortje,9.37,0,0,2.4,25,DC,2,0,50,0,0,6.0,0,56.0,0
1,11,1,0,AR Patel,4.00,0,0,4.0,16,DC,0,0,0,0,0,40.0,25,65.0,1
2,7,1,0,J Yadav,6.25,0,0,4.0,25,MI,1,0,25,0,0,24.0,10,59.0,2
3,8,3,0,JJ Bumrah,7.00,0,0,4.0,28,MI,0,0,0,0,0,20.0,10,30.0,3
4,6,5,1,K Rabada,10.66,0,0,3.0,32,DC,1,0,25,0,0,-2.0,-10,13.0,4
5,2,1,1,KA Pollard,13.00,0,0,1.0,13,MI,0,1,0,0,0,-2.0,0,-2.0,5
6,3,1,2,KH Pandya,10.00,0,0,3.0,30,MI,0,1,0,0,0,-4.0,-10,-14.0,6
7,3,1,2,MP Stoinis,11.50,0,0,2.0,23,DC,1,0,25,0,0,-4.0,-10,11.0,7
8,10,3,1,NM Coulter-Nile,7.25,0,0,4.0,29,MI,2,1,50,0,0,22.0,10,82.0,8
9,5,2,2,P Dubey,9.66,0,0,3.0,29,DC,0,0,0,0,0,0.0,0,0.0,9


In [9]:
for index, row in final_bowling_scorecard.iterrows():
    first, *middle, last = str(row["Player_Name"]).split()
    player_name_short=first[0].lower()+"%"+last.lower()
    all_queries_list.append("Update ipl_match_stats_new set bowling_pos="+str(row["bowling_pos"])+
           " where lower(player_name) like '"+player_name_short+"%' and team_name='"+row["Team_BOWL"]+
           "' and match_no="+str(match_no)+";")


for index, row in final_bowling_scorecard.iterrows():
    all_queries_list.append("Update ipl_match_stats_new set overs_bowled="+str(row["O"])+
           ",maidens="+str(row["M"])+
           ",runs_given="+str(row["R"])+
           ",wickets="+str(row["W"])+
           ",economy="+str(row["Econ"])+
           ",dots="+str(row["0s"])+
           ",fours="+str(row["4s"])+
           ",sixes="+str(row["6s"])+
           " where bowling_pos="+str(row["bowling_pos"])+" and match_no="+str(match_no)+";")


In [10]:
#fielding calculation
catches_list=[]
direct_run_out_list=[]
part_of_run_out_list=[]
stumpings_list=[]

#generate a smaller fielding input data frame from final batting scorecard
fielding_input=pd.DataFrame() 
fielding_input['Dismissal_Detail']=final_batting_scorecard['Dismissal_Detail']
fielding_input['Team_Field']=final_batting_scorecard['Team_Field']

#Generate 4 types of fielding scorecards (catches_list, direct run out, part of run out, stumpings)
for index, row in fielding_input.iterrows():
    test_string=row["Dismissal_Detail"]
    test_string=test_string.replace("retired hurt","not out")
    test_string=test_string.replace("hit wicket ","")
    ##Just for the 31st match
    test_string=test_string.replace("run out (Pandya)","run out (Hardik Pandya)")

    #team_commentary.append(test_string)
    st_pos=test_string.find('st ')

    if(("c & b") in test_string):
        test_string=test_string.replace("c & b","c")+test_string.replace("c & b"," b")
    if test_string[0]=="c" and test_string[1]==" ":
        test_string=test_string[2::]
    elif test_string[0]=="c" and test_string[1]!=" ":
        test_string=test_string[1::]
    bpos=test_string.rfind(' b ')
    if(bpos!=-1):
        test_string=test_string[0:bpos].strip()

    run_out_pos=test_string.find('run out (')

    if(test_string!='' and test_string!='lbw'  and test_string!='not out'):
        if(st_pos!=0) and (run_out_pos!=0) and not ("b ")in test_string:
            test_string=test_string.replace("†","").strip()
            catches_list.append(test_string+":::"+row['Team_Field'])
    if(run_out_pos==0 and '/' not in test_string):
        test_string=test_string.replace("run out (","")
        test_string=test_string.replace(")","")
        test_string=test_string.replace("†","").strip()
        direct_run_out=test_string
        if(direct_run_out_list!=''):
            direct_run_out_list.append(direct_run_out+":::"+row['Team_Field'])
    if(run_out_pos==0 and '/' in test_string):
        run_out_pos=test_string.find('run out (')
        if(run_out_pos==0):
            test_string=test_string.replace("run out (","")
            test_string=test_string.replace(")","")
        test_string=test_string.replace("†","").strip()
        part_of_run_out_guys=test_string.split("/")
        for part_of_run_out in part_of_run_out_guys:
            #print(part_of_run_out)
            if(part_of_run_out!='/'):
                part_of_run_out_list.append(part_of_run_out+":::"+row['Team_Field'])
                #print(part_of_run_out_list)
    if(st_pos==0):
        test_string=test_string[3::]
        test_string=test_string.replace("†","").strip()
        stumpings_list.append(test_string+":::"+row['Team_Field'])

In [11]:
#get final catches scorecard
if len(catches_list) > 0:
    catch_scorecard=pd.DataFrame(catches_list)
    catch_scorecard['Player_Name']= catch_scorecard[0].map(lambda v: v.split(':::')[0])
    catch_scorecard['Team_Field']= catch_scorecard[0].map(lambda v: v.split(':::')[1])
    del catch_scorecard[0]
    catch_scorecard=catch_scorecard.groupby(['Player_Name','Team_Field']).size().reset_index(name='Catches')
else:
    catch_scorecard=pd.DataFrame()


#get direct run out scorecard
if len(direct_run_out_list)>0:
    direct_run_out_scorecard=pd.DataFrame(direct_run_out_list)
    direct_run_out_scorecard['Player_Name']= direct_run_out_scorecard[0].map(lambda v: v.split(':::')[0])
    direct_run_out_scorecard['Team_Field']= direct_run_out_scorecard[0].map(lambda v: v.split(':::')[1])
    del direct_run_out_scorecard[0]
    direct_run_out_scorecard=direct_run_out_scorecard.groupby(['Player_Name','Team_Field']).size().reset_index(name='Direct_Run_Out')
else:
    direct_run_out_scorecard=pd.DataFrame()


#get part of run out scorecard
if len(part_of_run_out_list)>0:
    part_of_run_out_scorecard=pd.DataFrame(part_of_run_out_list)
    part_of_run_out_scorecard['Player_Name']= part_of_run_out_scorecard[0].map(lambda v: v.split(':::')[0])
    part_of_run_out_scorecard['Team_Field']= part_of_run_out_scorecard[0].map(lambda v: v.split(':::')[1])
    del part_of_run_out_scorecard[0]
    part_of_run_out_scorecard=part_of_run_out_scorecard.groupby(['Player_Name','Team_Field']).size().reset_index(name='Part_Of_Run_Out')
else:
    part_of_run_out_scorecard=pd.DataFrame()

#get stumpings scorecard
if len(stumpings_list)>0:
    stumpings_scorecard=pd.DataFrame(stumpings_list)
    stumpings_scorecard['Player_Name']= stumpings_scorecard[0].map(lambda v: v.split(':::')[0])
    stumpings_scorecard['Team_Field']= stumpings_scorecard[0].map(lambda v: v.split(':::')[1])
    del stumpings_scorecard[0]
    stumpings_scorecard=stumpings_scorecard.groupby(['Player_Name','Team_Field']).size().reset_index(name='Stumpings')
else:
    stumpings_scorecard=pd.DataFrame()

In [12]:
catch_scorecard

,Player_Name,Team_Field,Catches
0,Coulter-Nile,MI,1
1,HH Pandya,MI,1
2,Pant,DC,1
3,Rahane,DC,1
4,de Kock,MI,2


In [13]:
direct_run_out_scorecard

""


In [14]:
part_of_run_out_scorecard

,Player_Name,Team_Field,Part_Of_Run_Out
0,Coulter-Nile,MI,1
1,Dubey,DC,1
2,Pant,DC,1
3,SA Yadav,MI,1


In [15]:
stumpings_scorecard

""


In [16]:
def extract_player_name(player_name):
    player_name_list=player_name.split()
    player_first_name_part=""
    player_name_final="%"+player_name_list[-1]+"%"
    if len(player_name_list)>1:
        player_first_name_part="%"+player_name_list[0][0]+""
        player_name_final=player_first_name_part+player_name_final
        # only problem is Hardik and Krunal Pandya
        if("%h%pandya%" in player_name_final.lower()):
            player_name_final="h%pandya%"
        elif("%k%pandya%" in player_name_final.lower()):
            player_name_final="k%pandya%"
    player_name_final=player_name_final.lower().replace("sub(","").replace("(","").replace(")","")
    player_name_final=player_name_final.lower().replace("sub[","").replace("[","").replace("]","")

    return player_name_final

In [17]:
for index, row in catch_scorecard.iterrows():
    player_name_final=extract_player_name(str(row["Player_Name"]))
    all_queries_list.append("Update ipl_match_stats_new set catches="+str(row["Catches"])+
           " where lower(player_name) like '"+player_name_final+"' and team_name='"+row["Team_Field"]+
           "' and match_no="+str(match_no)+";")

In [18]:
for index, row in direct_run_out_scorecard.iterrows():
    player_name_final=extract_player_name(str(row["Player_Name"]))
    all_queries_list.append("Update ipl_match_stats_new set direct_run_out="+str(row["Direct_Run_Out"])+
           " where lower(player_name) like '"+player_name_final+"' and team_name='"+row["Team_Field"]+
           "' and match_no="+str(match_no)+";")

In [19]:
for index, row in part_of_run_out_scorecard.iterrows():
    player_name_final=extract_player_name(str(row["Player_Name"]))
    all_queries_list.append("Update ipl_match_stats_new set part_of_run_out="+str(row["Part_Of_Run_Out"])+
           " where lower(player_name) like '"+player_name_final+"' and team_name='"+row["Team_Field"]+
           "' and match_no="+str(match_no)+";")

In [20]:
for indexs, rows in stumpings_scorecard.iterrows():
    player_name_final=extract_player_name(str(rows["Player_Name"]))
    all_queries_list.append("Update ipl_match_stats_new set stumpings="+str(rows["Stumpings"])+
           " where lower(player_name) like '"+player_name_final+"' and team_name='"+rows["Team_Field"]+
           "' and match_no="+str(match_no)+";")

In [21]:
#for duck_out_player in duck_out_in0ball_players_list:
    #if(duck_out_player!=""):
        #all_queries_list.append("update ipl_match_stats_new  set is_out='Y' where match_no="+str(match_no)+"  and balls_faced =0 and lower(player_name) like lower('"+duck_out_player+"');")


In [22]:
#all_queries_list.append("update ipl_match_stats_new  set is_out='Y' where match_no="+str(match_no)+"  and balls_faced >0;")
all_queries_list.append("update ipl_match_stats_new  set strike_rate=ROUND((runs_scored*100/balls_faced),2) where match_no="+str(match_no)+" and balls_faced>0;")
all_queries_list.append("update ipl_match_stats_new  set economy=ROUND((runs_given/(floor(overs_bowled)+(overs_bowled-floor(overs_bowled))/6*10)),2) where match_no="+str(match_no)+"  and overs_bowled>0;")

all_queries_list.append("\n------------Manually update if any update has zero records updated add replace based on player name \n\n\n\n")
all_queries_list.append("\nupdate ipl_match_stats_new  set is_part_of_winning_team='Y' where match_no="+str(match_no)+"  and team_name='"+winning_team+"';") 
all_queries_list.append("update ipl_match_stats_new  set is_part_of_winning_team='N' where match_no="+str(match_no)+"  and team_name<>'"+winning_team+"';") 
all_queries_list.append("update ipl_match_stats_new  set is_mom='N' where match_no="+str(match_no)+" and player_name not like '%"+man_of_the_match+"%';") 
all_queries_list.append("update ipl_match_stats_new  set is_mom='Y' where match_no="+str(match_no)+" and player_name like '%"+man_of_the_match+"%';") 

#for not_out_player in not_out_in_0_players_list:
    #if(not_out_player!=""):
        #all_queries_list.append("update ipl_match_stats_new  set is_out='N' where match_no="+str(match_no)+"  and balls_faced >0 and lower(player_name) like lower('"+not_out_player+"');")
        


In [23]:
import numpy as np
import re
result = pd.concat([final_batting_scorecard.set_index('Player_Name'), final_bowling_scorecard.set_index('Player_Name')], axis=1,sort=True)
result=result.fillna(0)
result2=finaldata=pd.DataFrame()
result2['total_batting_points']=result['total_batting_points']
result2['total_bowling_points']=result['total_bowling_points']
result2['Team_BAT']=result['Team']
result2['Team_BOWL']=result['Team_BOWL']


conditions = [
    result2['Team_BAT']=='0', 
    result2['Team_BAT']!='0']

choices = [result2['Team_BOWL'], result2['Team_BAT']]

result2['Team'] = np.select(conditions, choices)


result2.loc[result2['Team']=='0','Team'] =result['Team_BOWL']

result2['total_bat_bowl_points']=result2['total_batting_points']+result2['total_bowling_points']
result2['Temp_Player_Name'] = result2.index
result2=result2.reset_index()
del result2['index']
result2.insert(0, 'Player_Name', result2['Temp_Player_Name'])
del result2['Temp_Player_Name']

result3=finaldata=pd.DataFrame()

result2_rows_list=[]

for index, row in result2.iterrows():
    if(row["Team_BAT"]==0):
        row["Team"]=row["Team_BOWL"]
    if(row["Team_BOWL"]==0):
        row["Team"]=row["Team_BAT"]
    row["catches_points"]=0
    row["part_of_run_out_points"]=0
    row["direct_run_out_points"]=0
    row["stumpings_points"]=0

    player_name=row["Player_Name"]
    for indexc, rowc in catch_scorecard.iterrows():
        pattern=extract_player_name(rowc["Player_Name"]).replace("%",".").replace("h.pandya.","hh.pandya.").replace("k.pandya.","kh.pandya.")
        compiled = re.compile(pattern)
        ms = compiled.search(player_name.lower()+" ")
        try:
            if(ms.string!=''):
                row["catches_points"]=rowc["Catches"]*5
                break
        except:
            pass
    
    for indexdr, rowdr in direct_run_out_scorecard.iterrows():
        pattern=extract_player_name(rowdr["Player_Name"]).replace("%",".").replace("h.pandya.","hh.pandya.").replace("k.pandya.","kh.pandya.")
        #print("pattern:::"+pattern)
        compiled = re.compile(pattern)
        ms = compiled.search(player_name.lower()+" ")
        try:
            if(ms.string!=''):
                #print(player_name)
                row["direct_run_out_points"]=rowdr["Direct_Run_Out"]*15
                break
        except:
            pass
    
    
    for indexpr, rowpr in part_of_run_out_scorecard.iterrows():
        pattern=extract_player_name(rowpr["Player_Name"]).replace("%",".").replace("h.pandya.","hh.pandya.").replace("k.pandya.","kh.pandya.")
        compiled = re.compile(pattern)
        ms = compiled.search(player_name.lower()+" ")
        try:
            if(ms.string!=''):
                row["part_of_run_out_points"]=rowpr["Part_Of_Run_Out"]*5
                break
        except:
            pass
    
    
    for indexs, rows in stumpings_scorecard.iterrows():
        pattern=extract_player_name(rows["Player_Name"]).replace("%",".").replace("h.pandya.","hh.pandya.").replace("k.pandya.","kh.pandya.")
        compiled = re.compile(pattern)
        ms = compiled.search(player_name.lower()+" ")
        try:
            if(ms.string!=''):
                row["stumpings_points"]=rows["Stumpings"]*10
                break
        except:
            pass
    
    row["Total_Batting_Points"]=row["total_batting_points"]
    row["Total_Bowling_Points"]=row["total_bowling_points"]

    row["Total_Fielding_Points"]=row["stumpings_points"]+row["part_of_run_out_points"]+row["direct_run_out_points"]+row["catches_points"]
    row["Total_Points"]=row["total_batting_points"]+row["total_bowling_points"]+row["Total_Fielding_Points"]
    
    result2_rows_list.append(row)
result3 = pd.DataFrame(result2_rows_list,)
del result3['Team_BAT']
del result3['Team_BOWL']
del result3['stumpings_points']
del result3['part_of_run_out_points']
del result3['direct_run_out_points']
del result3['catches_points']
del result3['total_bat_bowl_points']
del result3['total_batting_points']
del result3['total_bowling_points']


result3

,Player_Name,Team,Total_Batting_Points,Total_Bowling_Points,Total_Fielding_Points,Total_Points
0,A Nortje,DC,0.0,56.0,0,56.0
1,AM Rahane,DC,0.0,0.0,5,5.0
2,AR Patel,DC,5.0,65.0,0,70.0
3,HH Pandya,MI,0.0,0.0,5,5.0
4,Ishan Kishan,MI,68.0,0.0,0,68.0
5,J Yadav,MI,0.0,59.0,0,59.0
6,JJ Bumrah,MI,0.0,30.0,0,30.0
7,K Rabada,DC,-20.0,13.0,0,-7.0
8,KA Pollard,MI,10.0,-2.0,0,8.0
9,KH Pandya,MI,0.0,-14.0,0,-14.0


In [24]:
teams_list=teams.split("\t")
teams_list_len=len(teams_list)-1
person_count=int(teams_list_len/17)

match_no_list=[]
match_title_list=[]
person_name_list=[]
captain_list=[]
mom_list=[]
winner_list=[]
player_list=[]

for i in range(0, person_count):
    for j in range(0, 11):
        match_no_list.append(teams_list[(i*17)+0])
        match_title_list.append(teams_list[(i*17)+1])
        person_name_list.append(teams_list[(i*17)+2])
        captain_list.append(teams_list[(i*17)+14])
        mom_list.append(teams_list[(i*17)+15])
        winner_list.append(teams_list[(i*17)+16])

    player_list.append(teams_list[(i*17)+3])
    player_list.append(teams_list[(i*17)+4])
    player_list.append(teams_list[(i*17)+5])
    player_list.append(teams_list[(i*17)+6])
    player_list.append(teams_list[(i*17)+7])
    player_list.append(teams_list[(i*17)+8])
    player_list.append(teams_list[(i*17)+9])
    player_list.append(teams_list[(i*17)+10])
    player_list.append(teams_list[(i*17)+11])
    player_list.append(teams_list[(i*17)+12])
    player_list.append(teams_list[(i*17)+13])
team_selected_list=pd.DataFrame({'Match_No' : match_no_list,'Match_Title':match_title_list,
         'Person_Name' : person_name_list,
         'Player_Name': player_list,
         'Captain':captain_list,
         'Man_of_the_Match':mom_list,
         'winner':winner_list
         }, columns=['Match_No','Match_Title','Person_Name','Player_Name','Captain','Man_of_the_Match','winner'])
team_selected_list['Is_Captain']="N"
team_selected_list['Is_MoM']="N"
team_selected_list.loc[team_selected_list['Player_Name']==team_selected_list["Captain"],'Is_Captain'] = "Y"
team_selected_list.loc[team_selected_list['Player_Name']==team_selected_list["Man_of_the_Match"],'Is_MoM'] = "Y"
team_selected_list["points"]=0
del team_selected_list["Captain"]
del team_selected_list["Man_of_the_Match"]
#team_selected_list["Winner"]=team_selected_list["winner"]
#del team_selected_list["winner"]


In [25]:
for ii in range (1,3):
    teams_selected_fuzzy_list=[]
    for index1, row1 in team_selected_list.iterrows():
        prev_match_score=0
        my_player_name=""
        #print ("player_name1:"+player_name1)
        for index2, row2 in result3.iterrows():
            name_list_2 = row2["Player_Name"].split();
            if(row1["Player_Name"]=="Krunal Pandya"):
                row1["Player_Name"]="KH Pandya"
            elif(row1["Player_Name"]=="Hardik Pandya"):
                row1["Player_Name"]="HH Pandya"
            elif(row1["Player_Name"]=="Mujeeb Zadran"):
                row1["Player_Name"]="Mujeeb Ur Rahman"
            elif(row1["Player_Name"]=="Dinesh Karthik"):
                row1["Player_Name"]="KD Karthik"
            elif(row1["Player_Name"]=="Rashid Khan Arman"):
                row1["Player_Name"]="Rashid Khan"
            elif(row1["Player_Name"]=="Gowtham Krishnappa"):
                row1["Player_Name"]="Krishnappa Gowtham"
            
            name_list_1 = row1["Player_Name"].split();
            player_name1=name_list_1[0][0]+name_list_1[-1]

                
            player_name2=name_list_2[0][0]+name_list_2[-1]
            
            #if(player_name2=="CGayle"):
                #print(player_name2+"   "+player_name1)
            
    
    
            if(player_name2==player_name1):
                #print("****"+row2["Player_Name"]+"  "+player_name2)
                #if(player_name2=="KWilliamson"):
                    #print(str(row2["Total_Points"])+"  "+row2["Player_Name"])
                #row1["Old_Player_Name"]=row1["Player_Name"]
                player_name_new=row2["Player_Name"]
                row1["Player_Name"]=player_name_new
                row1["points"]=int(row2["Total_Points"])
                if(row1["Is_Captain"]=='Y'):
                    row1["points"]=int(row1["points"]*2)
            row1["Player_Name"]=row1["Player_Name"].replace("✈","")
        teams_selected_fuzzy_list.append(row1)
    
    teams_selected_fuzzy = pd.DataFrame(teams_selected_fuzzy_list)
    
    
    #del(teams_selected_fuzzy["Old_Player_Name"])
    del(teams_selected_fuzzy["Is_MoM"])
    del(teams_selected_fuzzy["Is_Captain"])
    
    
    
    teams_selected_fuzzy["points_1"]=teams_selected_fuzzy["points"]*-1
    teams_selected_fuzzy=teams_selected_fuzzy.sort_values(by=['Person_Name','points_1'])
    del(teams_selected_fuzzy["points_1"])
    teams_selected_fuzzy_pd=pd.DataFrame({'sum' : teams_selected_fuzzy.groupby( ['Person_Name'] )["points"].sum()}).reset_index()
    teams_selected_fuzzy_pd
    
    teams_selected_fuzzy_pd2=pd.DataFrame({'winner' : teams_selected_fuzzy.groupby( ['Person_Name'] )["winner"].max()}).reset_index()
    teams_selected_fuzzy_pd2["winner_points"]= -50
    teams_selected_fuzzy_pd2.loc[teams_selected_fuzzy_pd2["winner"]==winning_team, 'winner_points'] = 50
    
    teams_selected_fuzzy_pd3=pd.DataFrame()
    
    teams_selected_fuzzy_pd3 = pd.concat([teams_selected_fuzzy_pd.set_index('Person_Name'), teams_selected_fuzzy_pd2.set_index('Person_Name')], axis=1)
    del(teams_selected_fuzzy_pd3["winner"])
    teams_selected_fuzzy_pd3["total_points"]=teams_selected_fuzzy_pd3["sum"]+teams_selected_fuzzy_pd3["winner_points"]
    del(teams_selected_fuzzy_pd3["sum"])
    del(teams_selected_fuzzy_pd3["winner_points"])

    print("")

In [26]:
print("Player Points")
teams_selected_fuzzy=teams_selected_fuzzy.sort_values(by=['Person_Name'],ascending=True)
teams_selected_fuzzy

Player Points


,Match_No,Match_Title,Person_Name,Player_Name,winner,points
49,60,MI vs DC,Aditya,RG Sharma,MI,168
47,60,MI vs DC,Aditya,SS Iyer,MI,150
54,60,MI vs DC,Aditya,TA Boult,MI,134
44,60,MI vs DC,Aditya,Ishan Kishan,MI,68
50,60,MI vs DC,Aditya,Q de Kock,MI,55
46,60,MI vs DC,Aditya,SA Yadav,MI,48
45,60,MI vs DC,Aditya,S Dhawan,MI,30
51,60,MI vs DC,Aditya,JJ Bumrah,MI,30
48,60,MI vs DC,Aditya,R Ashwin,MI,28
53,60,MI vs DC,Aditya,K Rabada,MI,-7


In [27]:
print("Player Points#Summary")
teams_selected_fuzzy_pd=teams_selected_fuzzy_pd.sort_values(by=['sum'],ascending=False)
teams_selected_fuzzy_pd

Player Points#Summary


,Person_Name,sum
5,Shriyansh,720
2,Deepak,706
0,Aditya,695
1,Avani,597
6,Sowmya,505
3,Govardhan,429
4,Mitesh,421


In [28]:
print("Winner Team Points")
teams_selected_fuzzy_pd2

Winner Team Points


,Person_Name,winner,winner_points
0,Aditya,MI,50
1,Avani,MI,50
2,Deepak,MI,50
3,Govardhan,MI,50
4,Mitesh,DC,-50
5,Shriyansh,MI,50
6,Sowmya,MI,50


In [29]:
teams_selected_fuzzy_pd3
teams_selected_fuzzy_pd3=teams_selected_fuzzy_pd3.sort_values(by=['total_points'],ascending=False)
teams_selected_fuzzy_pd3

,total_points
Person_Name,
Shriyansh,770
Deepak,756
Aditya,745
Avani,647
Sowmya,555
Govardhan,479
Mitesh,371


In [30]:
team_selected_queries="'"+teams.replace('\t','\',\'')+"\n"
team_selected_queries=team_selected_queries.replace('\''+str(match_no)+'\'','\n'+'\''+str(match_no)+'\'').replace(',\n',');\ninsert into IPL_2020_TEAM_SELECTED values(');
team_selected_queries="insert into IPL_2020_TEAM_SELECTED values("+team_selected_queries[1::]
team_selected_queries=team_selected_queries.replace(",'\n",");")

print ("\n--Team Selected Queries below:\n"+team_selected_queries)

print("\n\n");
print(
"""--Insert player stats for the match
INSERT INTO ipl_2020_match_stats (match_no, match_title, MATCH_DATE, MATCH_TIME_IN_PM, team_name, player_name)
SELECT *FROM (SELECT              match_no, match_title, MATCH_DATE, MATCH_TIME_IN_PM, b.team,    player_name FROM IPL_2020_TEAM_PLAYER a, ipl_2020_match_view b WHERE     a.team_abbreviation = b.team AND a.player_name IN (SELECT player_name FROM IPL_2020_TEAM_PLAYER)                      
AND match_no IN ("""+str(match_no)+""") ORDER BY match_no);
            
COMMIT;

exec p_clean_ipl_2020_match_stats();

COMMIT;""")



--Team Selected Queries below:
insert into IPL_2020_TEAM_SELECTED values('60','MI vs DC','Govardhan','Surya Kumar Yadav','Ishan Kishan','Shikhar Dhawan','Shreyas Iyer','Kieron Pollard','Marcus Stoinis','Quinton De Kock','Jasprit Bumrah','Kagiso Rabada','Axar Patel','Krunal Pandya','Surya Kumar Yadav','Shikhar Dhawan','MI');
insert into IPL_2020_TEAM_SELECTED values('60','MI vs DC','Sowmya','Rohit Sharma','Shikhar Dhawan','Surya Kumar Yadav','Ishan Kishan','Kieron Pollard','Marcus Stoinis','Quinton De Kock','Kagiso Rabada','Jasprit Bumrah','Axar Patel','Rahul Chahar','Ishan Kishan','Jasprit Bumrah','MI');
insert into IPL_2020_TEAM_SELECTED values('60','MI vs DC','Avani','Shikhar Dhawan','Surya Kumar Yadav','Rohit Sharma','Shreyas Iyer','Marcus Stoinis','Axar Patel','Quinton De Kock','Kagiso Rabada','Jasprit Bumrah','Rahul Chahar','Anrich Nortje','Shikhar Dhawan','Jasprit Bumrah','MI');
insert into IPL_2020_TEAM_SELECTED values('60','MI vs DC','Deepak','Ishan Kishan','Shikhar Dhawan','S

In [31]:
all_queries_str="\n".join(all_queries_list)

all_queries_str=all_queries_str.replace("k%karthik","d%karthik")
#all_queries_str=all_queries_str.replace("k%gowtham","%gowtham%k%");
all_queries_str=all_queries_str.replace("lower(player_name) like 'm%rahman%' and team_name='KXIP'","lower(player_name) like 'mujeeb%' and team_name='KXIP'");
all_queries_str=all_queries_str.replace("klaasen%","kla%sen%");
all_queries_str=all_queries_str.replace("k%sharma%","karn%sharma%");
all_queries_str=all_queries_str.replace("m%krishna%","p%krishna%");
all_queries_str=all_queries_str.replace("c%varun%","v%chakravar%");
all_queries_str=all_queries_str.replace("lower(player_name) like 'p%singh%' and team_name='KXIP'","lower(player_name) like 's%singh%' and team_name='KXIP'");
all_queries_str=all_queries_str.replace("lower(player_name) like 'a%singh%' and team_name='KXIP'","lower(player_name) like 'arsh%singh%' and team_name='KXIP'");
all_queries_str=all_queries_str.replace("lower(player_name) like 'm%singh%' and team_name='KXIP'","lower(player_name) like 'man%singh%' and team_name='KXIP'");

all_queries_str=all_queries_str.replace("lower(player_name) like '%p%singh%' and team_name='KXIP'","lower(player_name) like 's%singh%' and team_name='KXIP'");
all_queries_str=all_queries_str.replace("lower(player_name) like '%a%singh%' and team_name='KXIP'","lower(player_name) like 'arsh%singh%' and team_name='KXIP'");
all_queries_str=all_queries_str.replace("lower(player_name) like '%m%singh%' and team_name='KXIP'","lower(player_name) like 'man%singh%' and team_name='KXIP'");
all_queries_str=all_queries_str.replace("lower(player_name) like '%a%patel%' and team_name='DC'","lower(player_name) like 'axar%patel%' and team_name='DC'");
all_queries_str=all_queries_str.replace("lower(player_name) like '%h%patel%' and team_name='DC'","lower(player_name) like 'har%patel%' and team_name='DC'");


all_queries_str=all_queries_str.replace("scantlebury-searles","searles");
all_queries_str=all_queries_str.replace("like '%sharma%' and team_name='CSK'","like 'karn%sharma%' and team_name='CSK'")
all_queries_str=all_queries_str.replace("ipl_match_stats_new","ipl_2020_match_stats")


print ("\n--Update queries below:\n"+all_queries_str)




--Update queries below:
Update ipl_2020_match_stats set batting_pos=2 where lower(player_name) like 'a%rahane%' and team_name='DC' and match_no=60;
Update ipl_2020_match_stats set batting_pos=6 where lower(player_name) like 'a%patel%' and team_name='DC' and match_no=60;
Update ipl_2020_match_stats set batting_pos=13 where lower(player_name) like 'h%pandya%' and team_name='MI' and match_no=60;
Update ipl_2020_match_stats set batting_pos=11 where lower(player_name) like 'i%kishan%' and team_name='MI' and match_no=60;
Update ipl_2020_match_stats set batting_pos=7 where lower(player_name) like 'k%rabada%' and team_name='DC' and match_no=60;
Update ipl_2020_match_stats set batting_pos=12 where lower(player_name) like 'k%pollard%' and team_name='MI' and match_no=60;
Update ipl_2020_match_stats set batting_pos=14 where lower(player_name) like 'k%pandya%' and team_name='MI' and match_no=60;
Update ipl_2020_match_stats set batting_pos=0 where lower(player_name) like 'm%stoinis%' and team_name=